# Project 3: Smart Beta Portfolio and Portfolio Optimization

## Overview


Smart beta has a broad meaning, but we can say in practice that when we use the universe of stocks from an index, and then apply some weighting scheme other than market cap weighting, it can be considered a type of smart beta fund.  A Smart Beta portfolio generally gives investors exposure or "beta" to one or more types of market characteristics (or factors) that are believed to predict prices while giving investors a diversified broad exposure to a particular market. Smart Beta portfolios generally target momentum, earnings quality, low volatility, and dividends or some combination. Smart Beta Portfolios are generally rebalanced infrequently and follow relatively simple rules or algorithms that are passively managed.  Model changes to these types of funds are also rare requiring prospectus filings with US Security and Exchange Commission in the case of US focused mutual funds or ETFs.. Smart Beta portfolios are generally long-only, they do not short stocks.

In contrast, a purely alpha-focused quantitative fund may use multiple models or algorithms to create a portfolio. The portfolio manager retains discretion in upgrading or changing the types of models and how often to rebalance the portfolio in attempt to maximize performance in comparison to a stock benchmark.  Managers may have discretion to short stocks in portfolios.

Imagine you're a portfolio manager, and wish to try out some different portfolio weighting methods.

One way to design portfolio is to look at certain accounting measures (fundamentals) that, based on past trends, indicate stocks that produce better results.  


For instance, you may start with a hypothesis that dividend-issuing stocks tend to perform better than stocks that do not. This may not always be true of all companies; for instance, Apple does not issue dividends, but has had good historical performance.  The hypothesis about dividend-paying stocks may go something like this: 

Companies that regularly issue dividends may also be more prudent in allocating their available cash, and may indicate that they are more conscious of prioritizing shareholder interests.  For example, a CEO may decide to reinvest cash into pet projects that produce low returns.  Or, the CEO may do some analysis, identify that reinvesting within the company produces lower returns compared to a diversified portfolio, and so decide that shareholders would be better served if they were given the cash (in the form of dividends).  So according to this hypothesis, dividends may be both a proxy for how the company is doing (in terms of earnings and cash flow), but also a signal that the company acts in the best interest of its shareholders.  Of course, it's important to test whether this works in practice.


You may also have another hypothesis, with which you wish to design a portfolio that can then be made into an ETF.  You may find that investors may wish to invest in passive beta funds, but wish to have less risk exposure (less volatility) in their investments.  The goal of having a low volatility fund that still produces returns similar to an index may be appealing to investors who have a shorter investment time horizon, and so are more risk averse.

So the objective of your proposed portfolio is to design a portfolio that closely tracks an index, while also minimizing the portfolio variance.  Also, if this portfolio can match the returns of the index with less volatility, then it has a higher risk-adjusted return (same return, lower volatility).

Smart Beta ETFs can be designed with both of these two general methods (among others): alternative weighting and minimum volatility ETF.


## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it to Udacity.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

The other packages that we're importing are `helper`, `project_helper`, and `project_tests`. These are custom packages built to help you solve the problems.  The `helper` and `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.
### Install Packages

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

### Load Packages

In [2]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests
import yfinance as yf

## Market Data
### Load Data
For this universe of stocks, we'll be selecting large dollar volume stocks. We're using this universe, since it is highly liquid.

In [3]:
df = pd.read_csv('../../data/project_3/eod-quotemedia.csv')

percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

### View Data
To see what one of these 2-d matrices looks like, let's take a look at the closing prices matrix.

In [4]:
project_helper.print_dataframe(close)

# Part 1: Smart Beta Portfolio
In Part 1 of this project, you'll build a portfolio using dividend yield to choose the portfolio weights. A portfolio such as this could be incorporated into a smart beta ETF.  You'll compare this portfolio to a market cap weighted index to see how well it performs. 

Note that in practice, you'll probably get the index weights from a data vendor (such as companies that create indices, like MSCI, FTSE, Standard and Poor's), but for this exercise we will simulate a market cap weighted index.

## Index Weights
The index we'll be using is based on large dollar volume stocks. Implement `generate_dollar_volume_weights` to generate the weights for this index. For each date, generate the weights based on dollar volume traded for that date. For example, assume the following is close prices and volume data:
```
                 Prices
               A         B         ...
2013-07-08     2         2         ...
2013-07-09     5         6         ...
2013-07-10     1         2         ...
2013-07-11     6         5         ...
...            ...       ...       ...

                 Volume
               A         B         ...
2013-07-08     100       340       ...
2013-07-09     240       220       ...
2013-07-10     120       500       ...
2013-07-11     10        100       ...
...            ...       ...       ...
```
The weights created from the function `generate_dollar_volume_weights` should be the following:
```
               A         B         ...
2013-07-08     0.126..   0.194..   ...
2013-07-09     0.759..   0.377..   ...
2013-07-10     0.075..   0.285..   ...
2013-07-11     0.037..   0.142..   ...
...            ...       ...       ...
```

In [8]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    #TODO: Implement function
    d_val = close * volume 
    print(d_val)
    print(d_val.sum(axis=1))
    val = d_val.divide(d_val.sum(axis=1), axis=0)

    return val

project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

                           JCB                PCVE                AIBZ
2015-08-24  348628075.71300000  608648315.28000009  300410784.98600000
2015-08-25 7575630049.36999989 6240087760.44999981 4380507847.72000027
2015-08-26  941144343.83999991  754203279.77999997  553581860.38199997
2015-08-27  364742751.09999996 5254224178.90000057  545086266.30700004
2015-08-24    1257687175.97900009
2015-08-25   18196225657.54000092
2015-08-26    2248929484.00199986
2015-08-27    6164053196.30700111
dtype: float64
Tests Passed


### View Data
Let's generate the index weights using `generate_dollar_volume_weights` and view them using a heatmap.

In [9]:
index_weights = generate_dollar_volume_weights(close, volume)
project_helper.plot_weights(index_weights, 'Index Weights')

ticker                    AAL                AAPL               ABBV  \
date                                                                   
2013-07-01 202391976.65295082 5192133342.44938660 160240491.76812646   
2013-07-02 170044135.53290918 6379858749.09848881 154675436.02818424   
2013-07-03 113535574.39577270 3289403473.24362564  85687954.74287605   
2013-07-05 104208202.16722879 3711212490.37529945  98378473.62004057   
2013-07-08 116808737.36893333 4014865794.27665520 145851994.43971246   
...                       ...                 ...                ...   
2017-06-26 193104375.22673279 3688721869.74175406 282789387.32174462   
2017-06-27 205052732.54046640 3504176995.42216110 314414821.76263034   
2017-06-28 207995036.87223029 3170652004.96980381 295954447.98199576   
2017-06-29 303539868.45206839 4456079810.40160465 311591046.10151750   
2017-06-30 330201450.72510487 3264828722.90565109 331668224.65941626   

ticker                    ABT                AGN               

## Portfolio Weights
Now that we have the index weights, let's choose the portfolio weights based on dividend. You would normally calculate the weights based on trailing dividend yield, but we'll simplify this by just calculating the total dividend yield over time.

Implement `calculate_dividend_weights` to return the weights for each stock based on its total dividend yield over time. This is similar to generating the weight for the index, but it's using dividend data instead.
For example, assume the following is `dividends` data:
```
                 Prices
               A         B
2013-07-08     0         0
2013-07-09     0         1
2013-07-10     0.5       0
2013-07-11     0         0
2013-07-12     2         0
...            ...       ...
```
The weights created from the function `calculate_dividend_weights` should be the following:
```
               A         B
2013-07-08     NaN       NaN
2013-07-09     0         1
2013-07-10     0.333..   0.666..
2013-07-11     0.333..   0.666..
2013-07-12     0.714..   0.285..
...            ...       ...
```

In [12]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    #TODO: Implement function
    print(dividends)
    div_cumsum = dividends.cumsum()
    print(div_cumsum)
    dividend_weights = div_cumsum.divide(div_cumsum.sum(axis=1), axis=0)
    print(dividend_weights)

    return dividend_weights

project_tests.test_calculate_dividend_weights(calculate_dividend_weights)

                 HBUK       EEMD       STUM
2015-02-24 0.00000000 0.00000000 0.00000000
2015-02-25 0.00000000 0.00000000 0.10000000
2015-02-26 0.00000000 1.00000000 0.30000000
2015-02-27 0.00000000 0.20000000 0.00000000
                 HBUK       EEMD       STUM
2015-02-24 0.00000000 0.00000000 0.00000000
2015-02-25 0.00000000 0.00000000 0.10000000
2015-02-26 0.00000000 1.00000000 0.40000000
2015-02-27 0.00000000 1.20000000 0.40000000
                 HBUK       EEMD       STUM
2015-02-24        nan        nan        nan
2015-02-25 0.00000000 0.00000000 1.00000000
2015-02-26 0.00000000 0.71428571 0.28571429
2015-02-27 0.00000000 0.75000000 0.25000000
Tests Passed


### View Data
Just like the index weights, let's generate the ETF weights and view them using a heatmap.

In [13]:
etf_weights = calculate_dividend_weights(dividends)
project_helper.plot_weights(etf_weights, 'ETF Weights')

ticker            AAL       AAPL       ABBV        ABT        AGN        AIG  \
date                                                                           
2013-07-01 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-02 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-03 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-05 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-08 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
...               ...        ...        ...        ...        ...        ...   
2017-06-26 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2017-06-27 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2017-06-28 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2017-06-29 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2017-06-30 0.00000000 0.00000000 0.00000

## Returns
Implement `generate_returns` to generate returns data for all the stocks and dates from price data. You might notice we're implementing returns and not log returns. Since we're not dealing with volatility, we don't have to use log returns.

In [18]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    #TODO: Implement function
    print(prices)
    returns = (prices - prices.shift(1))/prices.shift(1)
    print(returns)

    return returns

project_tests.test_generate_returns(generate_returns)

                  LYGU        JHHI         LBM
2006-04-24 35.44110000 34.17990000 34.02230000
2006-04-25 92.11310000 91.05430000 90.95720000
2006-04-26 57.97080000 57.78140000 58.19820000
2006-04-27 34.17050000 92.45300000 58.51070000
                  LYGU        JHHI         LBM
2006-04-24         nan         nan         nan
2006-04-25  1.59904743  1.66397210  1.67345829
2006-04-26 -0.37065629 -0.36541822 -0.36015840
2006-04-27 -0.41055669  0.60004777  0.00536958
Tests Passed


### View Data
Let's generate the closing returns using `generate_returns` and view them using a heatmap.

In [19]:
returns = generate_returns(close)
project_helper.plot_returns(returns, 'Close Returns')

ticker             AAL         AAPL        ABBV         ABT          AGN  \
date                                                                       
2013-07-01 16.17609308  53.10917319 34.92447839 31.42538772 122.62751990   
2013-07-02 15.81983388  54.31224742 35.42807578 31.27288084 121.05361758   
2013-07-03 16.12794994  54.61204262 35.44486235 30.72565028 121.21003024   
2013-07-05 16.21460758  54.17338125 35.85613355 31.32670680 123.53666845   
2013-07-08 16.31089385  53.86579916 36.66188936 31.76628544 123.65397794   
...                ...          ...         ...         ...          ...   
2017-06-26 48.36234805 143.57270901 70.35520945 47.95227368 244.94985746   
2017-06-27 48.08474540 141.51491885 70.01668424 47.71697322 242.17472349   
2017-06-28 48.82832394 143.58255490 70.52930812 47.53069368 242.78485932   
2017-06-29 49.19515602 141.46568942 70.10373358 47.77579833 240.13765706   
2017-06-30 49.88916265 141.80044954 70.13275003 47.65814810 239.22245330   

ticker     

## Weighted Returns
With the returns of each stock computed, we can use it to compute the returns for an index or ETF. Implement `generate_weighted_returns` to create weighted returns using the returns and weights.

In [21]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    #TODO: Implement function
    weight_returns = returns * weights
    print(weight_returns)

    return weight_returns

project_tests.test_generate_weighted_returns(generate_weighted_returns)

                  DOFQ         GNO         YEW
2005-05-09         nan         nan         nan
2005-05-10  1.31241616  0.80759119  1.26836009
2005-05-11 -0.03779367 -0.02143549 -0.03449679
2005-05-12 -0.01622375  0.24521625  0.00050642
Tests Passed


### View Data
Let's generate the ETF and index returns using `generate_weighted_returns` and view them using a heatmap.

In [22]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.00008007  0.00308984  0.00004768 -0.00001880 -0.00003423   
2013-07-03  0.00009093  0.00074662  0.00000167 -0.00018529  0.00000450   
2013-07-05  0.00001716 -0.00091335  0.00003497  0.00008758  0.00005552   
2013-07-08  0.00001590 -0.00052257  0.00007514  0.00007710  0.00000238   
...                ...         ...         ...         ...         ...   
2017-06-26  0.00001016 -0.00019778  0.00000664 -0.00001502  0.00007702   
2017-06-27 -0.00001813 -0.00077347 -0.00002330 -0.00001404 -0.00007743   
2017-06-28  0.00004971  0.00071601  0.00003349 -0.00002399  0.00001622   
2017-06-29  0.00002938 -0.00084636 -0.00002422  0.00001920 -0.00006350   
2017-06-30  0.00007130  0.00011825  0.00000210 -0.00000848 -0.00002407   

ticker             AIG        AMAT   

## Cumulative Returns
To compare performance between the ETF and Index, we're going to calculate the tracking error. Before we do that, we first need to calculate the index and ETF comulative returns. Implement `calculate_cumulative_returns` to calculate the cumulative returns over time given the returns.

In [41]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    #TODO: Implement function
    print(returns)
    he = returns.sum(axis=1, skipna=False) + 1
    print(he)
    he = he.cumprod()
    print(he)

    return he

project_tests.test_calculate_cumulative_returns(calculate_cumulative_returns)

                  YKRR        HKNL        JGUP
2009-08-07         nan         nan         nan
2009-08-08  1.59904743  1.66397210  1.67345829
2009-08-09 -0.37065629 -0.36541822 -0.36015840
2009-08-10 -0.41055669  0.60004777  0.00536958
2009-08-07           nan
2009-08-08    5.93647782
2009-08-09   -0.09623291
2009-08-10    1.19486066
dtype: float64
2009-08-07           nan
2009-08-08    5.93647782
2009-08-09   -0.57128454
2009-08-10   -0.68260542
dtype: float64
Tests Passed


### View Data
Let's generate the ETF and index cumulative returns using `calculate_cumulative_returns` and compare the two.

In [42]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns, 'Smart Beta ETF vs Index')

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.00008007  0.00308984  0.00004768 -0.00001880 -0.00003423   
2013-07-03  0.00009093  0.00074662  0.00000167 -0.00018529  0.00000450   
2013-07-05  0.00001716 -0.00091335  0.00003497  0.00008758  0.00005552   
2013-07-08  0.00001590 -0.00052257  0.00007514  0.00007710  0.00000238   
...                ...         ...         ...         ...         ...   
2017-06-26  0.00001016 -0.00019778  0.00000664 -0.00001502  0.00007702   
2017-06-27 -0.00001813 -0.00077347 -0.00002330 -0.00001404 -0.00007743   
2017-06-28  0.00004971  0.00071601  0.00003349 -0.00002399  0.00001622   
2017-06-29  0.00002938 -0.00084636 -0.00002422  0.00001920 -0.00006350   
2017-06-30  0.00007130  0.00011825  0.00000210 -0.00000848 -0.00002407   

ticker             AIG        AMAT   

## Tracking Error
In order to check the performance of the smart beta portfolio, we can calculate the annualized tracking error against the index. Implement `tracking_error` to return the tracking error between the ETF and benchmark.

For reference, we'll be using the following annualized tracking error function:
$$ TE = \sqrt{252} * SampleStdev(r_p - r_b) $$

Where $ r_p $ is the portfolio/ETF returns and $ r_b $ is the benchmark returns.

_Note: When calculating the sample standard deviation, the delta degrees of freedom is 1, which is the also the default value._

In [44]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    #TODO: Implement function
    te = np.sqrt(252) * np.std(etf_returns_by_date - benchmark_returns_by_date, ddof=1)

    return te

project_tests.test_tracking_error(tracking_error)

Tests Passed


### View Data
Let's generate the tracking error using `tracking_error`.

In [45]:
smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

Smart Beta Tracking Error: 0.10202570304150263


# Part 2: Portfolio Optimization

Now, let's create a second portfolio.  We'll still reuse the market cap weighted index, but this will be independent of the dividend-weighted portfolio that we created in part 1.

We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.

Why are we doing this? One way that investors evaluate a fund is by how well it tracks its index. The fund is still expected to deviate from the index within a certain range in order to improve fund performance.  A way for a fund to track the performance of its benchmark is by keeping its asset weights similar to the weights of the index.  We’d expect that if the fund has the same stocks as the benchmark, and also the same weights for each stock as the benchmark, the fund would yield about the same returns as the benchmark. By minimizing a linear combination of both the portfolio risk and distance between portfolio and benchmark weights, we attempt to balance the desire to minimize portfolio variance with the goal of tracking the index.


## Covariance
Implement `get_covariance_returns` to calculate the covariance of the `returns`. We'll use this to calculate the portfolio variance.

If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [`Numpy.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time. For any `NaN` values, you can replace them with zeros using the [`DataFrame.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) function.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

In [54]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    #TODO: Implement function
    ret = returns.fillna(0)
    print(ret)
    cov = np.cov(ret.T)
    print(cov)
    
    return cov

project_tests.test_get_covariance_returns(get_covariance_returns)

                   CIR        CEQE         HPZ
2002-08-27  0.00000000  0.00000000  0.00000000
2002-08-28  1.59904743  1.66397210  1.67345829
2002-08-29 -0.37065629 -0.36541822 -0.36015840
2002-08-30 -0.41055669  0.60004777  0.00536958
[[0.89856076 0.7205586  0.8458721 ]
 [0.7205586  0.78707297 0.76450378]
 [0.8458721  0.76450378 0.83182775]]
Tests Passed


### View Data
Let's look at the covariance generated from `get_covariance_returns`.

In [55]:
covariance_returns = get_covariance_returns(returns)
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)

project_helper.plot_covariance_returns_correlation(
    covariance_returns_correlation,
    'Covariance Returns Correlation Matrix')

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000   
2013-07-02 -0.02202381  0.02265285  0.01441961 -0.00485298 -0.01283482   
2013-07-03  0.01947657  0.00551985  0.00047382 -0.01749857  0.00129209   
2013-07-05  0.00537313 -0.00803232  0.01160313  0.01956204  0.01919510   
2013-07-08  0.00593824 -0.00567773  0.02247191  0.01403207  0.00094959   
...                ...         ...         ...         ...         ...   
2017-06-26  0.00308452 -0.00314465  0.00137665 -0.00346373  0.00994076   
2017-06-27 -0.00574006 -0.01433274 -0.00481166 -0.00490697 -0.01132940   
2017-06-28  0.01546392  0.01461073  0.00732145 -0.00390384  0.00251940   
2017-06-29  0.00751269 -0.01474319 -0.00603401  0.00515677 -0.01090349   
2017-06-30  0.01410721  0.00236637  0.00041391 -0.00246255 -0.00381116   

ticker             AIG        AMAT   

KeyError: 'xaxis2'

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
Recall from the Pythagorean theorem that you can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. 

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|_2$


This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$).

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  Can you guess what to pass into this function?



### constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

In [63]:
import cvxpy as cvx

def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    assert len(covariance_returns.shape) == 2
    assert len(index_weights.shape) == 1
    assert covariance_returns.shape[0] == covariance_returns.shape[1]  == index_weights.shape[0]

    #TODO: Implement function
    
    x = cvx.Variable(len(index_weights))
    contraints = [x >=0, sum(x) == 1]
    print(x)
    print(contraints)

    covVariance = cvx.quad_form(x, covariance_returns)
    print(covVariance)
    index_dist = cvx.norm(x - index_weights, p=2)
    print(index_dist)
    objective = cvx.Minimize(covVariance + scale * index_dist)
    print(objective)

    problem = cvx.Problem(objective, contraints)
    problem.solve()

    xVal = x.value

    return xVal

project_tests.test_get_optimal_weights(get_optimal_weights)

var622
[Inequality(Constant(CONSTANT, ZERO, ())), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]
QuadForm(var622, [[0.143123   0.0216755  0.014273  ]
 [0.0216755  0.0401826  0.00663152]
 [0.014273   0.00663152 0.044963  ]])
Pnorm(var622 + -[0.23623892 0.0125628  0.7511982 ], 2)
minimize QuadForm(var622, [[0.143123   0.0216755  0.014273  ]
 [0.0216755  0.0401826  0.00663152]
 [0.014273   0.00663152 0.044963  ]]) + 2.0 * Pnorm(var622 + -[0.23623892 0.0125628  0.7511982 ], 2)
Tests Passed


## Optimized Portfolio
Using the `get_optimal_weights` function, let's generate the optimal ETF weights without rebalanceing. We can do this by feeding in the covariance of the entire history of data. We also need to feed in a set of index weights. We'll go with the average weights of the index over time.

In [64]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, index_weights.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)

var723
[Inequality(Constant(CONSTANT, ZERO, ())), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]
QuadForm(var723, [[5.31791451e-04 7.21251856e-05 9.55847963e-05 ... 4.55526842e-05
  1.26756746e-04 4.27885833e-05]
 [7.21251856e-05 2.14185487e-04 4.87000321e-05 ... 3.43023358e-05
  8.17766960e-05 4.13786801e-05]
 [9.55847963e-05 4.87000321e-05 2.64537496e-04 ... 3.32734532e-05
  1.11616723e-04 5.40669112e-05]
 ...
 [4.55526842e-05 3.43023358e-05 3.32734532e-05 ... 1.14424083e-04
  3.48568802e-05 2.79122492e-05]
 [1.26756746e-04 8.17766960e-05 1.11616723e-04 ... 3.48568802e-05
  7.17162013e-04 8.42633668e-05]
 [4.27885833e-05 4.13786801e-05 5.40669112e-05 ... 2.79122492e-05
  8.42633668e-05 1.29561995e-04]])
Pnorm(var723 + -[0.00505383 0.04996914 0.00507628 0.00344529 0.00631443 0.00723481
 0.00628914 0.00690689 0.05022972 0.00367206 0.00923578 0.00420389
 0.00659224 0.03065369 0.0057022  0.00386121 0.0159118  0.00755899
 0.00230404 0.00784448 0.00719021 0

With our ETF weights built, let's compare it to the index. Run the next cell to calculate the ETF returns and compare it to the index returns.

In [65]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, optim_etf_cumulative_returns, 'Optimized ETF vs Index')

optim_etf_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.00011130  0.00113194  0.00007320 -0.00001672 -0.00008104   
2013-07-03  0.00009843  0.00027582  0.00000241 -0.00006029  0.00000816   
2013-07-05  0.00002715 -0.00040137  0.00005890  0.00006740  0.00012121   
2013-07-08  0.00003001 -0.00028371  0.00011407  0.00004834  0.00000600   
...                ...         ...         ...         ...         ...   
2017-06-26  0.00001559 -0.00015714  0.00000699 -0.00001193  0.00006277   
2017-06-27 -0.00002901 -0.00071619 -0.00002443 -0.00001691 -0.00007154   
2017-06-28  0.00007815  0.00073009  0.00003717 -0.00001345  0.00001591   
2017-06-29  0.00003797 -0.00073670 -0.00003063  0.00001777 -0.00006885   
2017-06-30  0.00007130  0.00011825  0.00000210 -0.00000848 -0.00002407   

ticker             AIG        AMAT   

Optimized ETF Tracking Error: 0.05792147628283928


## Rebalance Portfolio Over Time
The single optimized ETF portfolio used the same weights for the entire history. This might not be the optimal weights for the entire period. Let's rebalance the portfolio over the same period instead of using the same weights. Implement `rebalance_portfolio` to rebalance a portfolio.

Reblance the portfolio every n number of days, which is given as `shift_size`. When rebalancing, you should look back a certain number of days of data in the past, denoted as `chunk_size`. Using this data, compute the optoimal weights using `get_optimal_weights` and `get_covariance_returns`.

In [66]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    #TODO: Implement function
    
    all_rebalance_weights = []
    for i in range(chunk_size, len(returns), shift_size):
        chunks = returns.iloc[i - chunk_size:i]
        cov_returns = get_covariance_returns(chunks)
        opt_weights = get_optimal_weights(cov_returns, index_weights.iloc[i-1])
        all_rebalance_weights.append(opt_weights)

    return all_rebalance_weights

project_tests.test_rebalance_portfolio(rebalance_portfolio)

                   VEU         RVS       WJHY
2010-05-17  0.00000000  0.00000000 0.00000000
2010-05-18 -0.02202381  0.02265285 0.01441961
2010-05-19  0.01947657  0.00551985 0.00047382
2010-05-20  0.00537313 -0.00803232 0.01160313
[[ 2.97086366e-04 -1.49593881e-04 -8.82401092e-05]
 [-1.49593881e-04  1.68909932e-04  3.42159759e-05]
 [-8.82401092e-05  3.42159759e-05  5.57551203e-05]]
var1112
[Inequality(Constant(CONSTANT, ZERO, ())), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]
QuadForm(var1112, [[ 2.97086366e-04 -1.49593881e-04 -8.82401092e-05]
 [-1.49593881e-04  1.68909932e-04  3.42159759e-05]
 [-8.82401092e-05  3.42159759e-05  5.57551203e-05]])
Pnorm(var1112 + -[0.00395679 0.1243466  0.00335064], 2)
minimize QuadForm(var1112, [[ 2.97086366e-04 -1.49593881e-04 -8.82401092e-05]
 [-1.49593881e-04  1.68909932e-04  3.42159759e-05]
 [-8.82401092e-05  3.42159759e-05  5.57551203e-05]]) + 2.0 * Pnorm(var1112 + -[0.00395679 0.1243466  0.00335064], 2)
          

Run the following cell to rebalance the portfolio using `rebalance_portfolio`.

In [67]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(returns, index_weights, shift_size, chunk_size)

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000   
2013-07-02 -0.02202381  0.02265285  0.01441961 -0.00485298 -0.01283482   
2013-07-03  0.01947657  0.00551985  0.00047382 -0.01749857  0.00129209   
2013-07-05  0.00537313 -0.00803232  0.01160313  0.01956204  0.01919510   
2013-07-08  0.00593824 -0.00567773  0.02247191  0.01403207  0.00094959   
...                ...         ...         ...         ...         ...   
2014-06-20  0.03460288 -0.01034182 -0.01642369 -0.00365854  0.00761905   
2014-06-23 -0.00426487 -0.00087999  0.00844278 -0.00293758  0.02056342   
2014-06-24 -0.02662308 -0.00605527 -0.00297674 -0.00687454  0.01292071   
2014-06-25  0.01692953  0.00088613  0.02631088  0.00618047 -0.00615494   
2014-06-26  0.01457527  0.00597610  0.02745455 -0.00196560  0.00565453   

ticker             AIG        AMAT   

[250 rows x 99 columns]
[[ 5.26624199e-04 -1.35807963e-05  7.62997033e-05 ...  2.55169552e-05
   1.30993487e-04  2.22766729e-05]
 [-1.35807963e-05  2.19743915e-04  1.12888037e-05 ...  1.36259233e-05
   3.43260436e-05 -4.73731655e-06]
 [ 7.62997033e-05  1.12888037e-05  2.13634859e-04 ...  2.93989616e-05
   7.30896142e-05  3.22391845e-05]
 ...
 [ 2.55169552e-05  1.36259233e-05  2.93989616e-05 ...  4.96201856e-05
   1.99587362e-05  1.87088238e-05]
 [ 1.30993487e-04  3.43260436e-05  7.30896142e-05 ...  1.99587362e-05
   3.66077280e-04  2.54605726e-05]
 [ 2.22766729e-05 -4.73731655e-06  3.22391845e-05 ...  1.87088238e-05
   2.54605726e-05  7.27960027e-05]]
var2683
[Inequality(Constant(CONSTANT, ZERO, ())), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]
QuadForm(var2683, [[ 5.26624199e-04 -1.35807963e-05  7.62997033e-05 ...  2.55169552e-05
   1.30993487e-04  2.22766729e-05]
 [-1.35807963e-05  2.19743915e-04  1.12888037e-05 ...  1.36259233e-05
   3.43260436e-0

[[ 5.42276537e-04 -1.52701443e-05  7.80751921e-05 ...  3.09490405e-05
   1.27796025e-04  2.68196735e-05]
 [-1.52701443e-05  2.08553922e-04  1.82872144e-05 ...  1.59317586e-05
   3.41117920e-05  3.54131680e-06]
 [ 7.80751921e-05  1.82872144e-05  2.15860014e-04 ...  3.15702481e-05
   8.06367200e-05  3.84583322e-05]
 ...
 [ 3.09490405e-05  1.59317586e-05  3.15702481e-05 ...  5.21174961e-05
   1.82960184e-05  2.33751013e-05]
 [ 1.27796025e-04  3.41117920e-05  8.06367200e-05 ...  1.82960184e-05
   3.88837119e-04  3.67370584e-05]
 [ 2.68196735e-05  3.54131680e-06  3.84583322e-05 ...  2.33751013e-05
   3.67370584e-05  8.24081620e-05]]
var3850
[Inequality(Constant(CONSTANT, ZERO, ())), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]
QuadForm(var3850, [[ 5.42276537e-04 -1.52701443e-05  7.80751921e-05 ...  3.09490405e-05
   1.27796025e-04  2.68196735e-05]
 [-1.52701443e-05  2.08553922e-04  1.82872144e-05 ...  1.59317586e-05
   3.41117920e-05  3.54131680e-06]
 [ 7.

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-09-04  0.02562538  0.02069467  0.01267903  0.01609475  0.01504308   
2013-09-05  0.01070791 -0.00685996  0.00881057  0.00836820  0.00762590   
2013-09-06 -0.01118305  0.00595635  0.00873362 -0.00711322 -0.01906326   
2013-09-09  0.03988095  0.01595681  0.01230349  0.00895522 -0.01084504   
2013-09-10  0.03720664 -0.02277891  0.00787756  0.00857988 -0.00161884   
...                ...         ...         ...         ...         ...   
2014-08-25 -0.01079046  0.00217134  0.01213768  0.00902398 -0.00307949   
2014-08-26 -0.00837139 -0.00641127 -0.00214784  0.00635444  0.00489828   
2014-08-27 -0.00255820  0.01230065 -0.00089686 -0.00374181 -0.00619181   
2014-08-28  0.00461657  0.00117497 -0.00179533 -0.00586854  0.00123724   
2014-08-29 -0.00737810  0.00244499 -0.00575540 -0.00259740  0.00172117   

ticker             AIG        AMAT   

[[4.72230777e-04 1.34552362e-05 6.20735125e-05 ... 2.09131636e-05
  1.21986960e-04 2.17773948e-05]
 [1.34552362e-05 1.72912966e-04 2.28435888e-05 ... 1.40264090e-05
  3.20014290e-05 6.18530435e-06]
 [6.20735125e-05 2.28435888e-05 2.02607531e-04 ... 2.68422849e-05
  7.90243601e-05 3.48047807e-05]
 ...
 [2.09131636e-05 1.40264090e-05 2.68422849e-05 ... 4.87128844e-05
  1.64742910e-05 2.27849875e-05]
 [1.21986960e-04 3.20014290e-05 7.90243601e-05 ... 1.64742910e-05
  4.00136869e-04 3.81968458e-05]
 [2.17773948e-05 6.18530435e-06 3.48047807e-05 ... 2.27849875e-05
  3.81968458e-05 7.96469314e-05]]
var6184
[Inequality(Constant(CONSTANT, ZERO, ())), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]
QuadForm(var6184, [[4.72230777e-04 1.34552362e-05 6.20735125e-05 ... 2.09131636e-05
  1.21986960e-04 2.17773948e-05]
 [1.34552362e-05 1.72912966e-04 2.28435888e-05 ... 1.40264090e-05
  3.20014290e-05 6.18530435e-06]
 [6.20735125e-05 2.28435888e-05 2.02607531e-04 ... 2.

...

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2016-04-26 -0.00216216 -0.00694709 -0.00065606 -0.00295387 -0.02183678   
2016-04-27 -0.00568797 -0.06257786 -0.00377482 -0.00113947  0.00059107   
2016-04-28 -0.02015800 -0.03056635  0.00823723 -0.07780059 -0.00554369   
2016-04-29 -0.03558521 -0.01149425 -0.00326797 -0.03760515 -0.01046379   
2016-05-02 -0.00461228 -0.00106678  0.00721311 -0.00128535  0.00715737   
...                ...         ...         ...         ...         ...   
2017-04-17  0.02029988  0.00552995  0.00280680  0.01499883 -0.00714796   
2017-04-18 -0.00949582 -0.00444194 -0.01026279  0.00277072 -0.00741117   
2017-04-19  0.01346724 -0.00368272 -0.00314218  0.00368409  0.00836249   
2017-04-20  0.01846847  0.01251066  0.00520095  0.00894701  0.00676991   
2017-04-21 -0.00154799 -0.00119348  0.00062716 -0.01023192 -0.00668236   

ticker             AIG        AMAT   

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2016-05-10  0.02519733  0.00678952  0.00062794  0.00315956  0.05282860   
2016-05-11 -0.02694699 -0.00974095 -0.01631628 -0.00236220 -0.01017778   
2016-05-12 -0.04656117 -0.02345692 -0.00334928 -0.00026309 -0.03003906   
2016-05-13  0.00574529  0.00199247 -0.00784125 -0.01052632  0.03393204   
2016-05-16  0.01935893  0.03711887  0.00725806  0.01728723  0.00747705   
...                ...         ...         ...         ...         ...   
2017-05-01 -0.01267011  0.02039680  0.00818926 -0.00366636  0.00561798   
2017-05-02  0.04253802  0.00634466  0.00225632  0.00275989  0.00110101   
2017-05-03  0.00113973 -0.00305064  0.00165091  0.00275229 -0.00708758   
2017-05-04 -0.00022769 -0.00360397  0.00734192  0.01989936  0.00611257   
2017-05-05  0.01366431  0.01658363 -0.00356983  0.00112133 -0.00126402   

ticker             AIG        AMAT   

[Inequality(Constant(CONSTANT, ZERO, ())), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]
QuadForm(var58310, [[5.15979087e-04 5.99130566e-05 5.37692882e-05 ... 1.00973424e-05
  1.27496273e-04 4.80072827e-05]
 [5.99130566e-05 1.26671702e-04 2.23360162e-05 ... 1.37214913e-05
  4.40691053e-05 1.51858781e-05]
 [5.37692882e-05 2.23360162e-05 1.62041991e-04 ... 2.18591977e-05
  4.90094973e-05 2.78432693e-05]
 ...
 [1.00973424e-05 1.37214913e-05 2.18591977e-05 ... 7.73675078e-05
  2.20249946e-06 8.00879812e-06]
 [1.27496273e-04 4.40691053e-05 4.90094973e-05 ... 2.20249946e-06
  5.33857872e-04 3.92536284e-05]
 [4.80072827e-05 1.51858781e-05 2.78432693e-05 ... 8.00879812e-06
  3.92536284e-05 9.03226428e-05]])
Pnorm(var58310 + -[0.00428806 0.05768555 0.00606241 0.00353434 0.00905647 0.00820911
 0.01443284 0.00556686 0.05412972 0.00428015 0.01156565 0.00351575
 0.01014423 0.02871557 0.00706727 0.00641229 0.01726997 0.00659065
 0.00349312 0.00513491 0.00861163 0.01

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2016-06-15  0.00428195 -0.00328340  0.00834725 -0.00610728 -0.02031954   
2016-06-16 -0.04427681  0.00422071  0.00827815  0.00187016 -0.00773605   
2016-06-17  0.00686342 -0.02275756 -0.01461412 -0.00293333 -0.00943993   
2016-06-20  0.01261077 -0.00241267  0.00166639  0.00962824  0.00089343   
2016-06-21  0.01043420  0.00851735 -0.00216270 -0.00052980 -0.01576979   
...                ...         ...         ...         ...         ...   
2017-06-06  0.00000000  0.00337816  0.00533254 -0.00557701  0.00056973   
2017-06-07  0.02251709  0.00595662  0.01296596  0.00431406 -0.00052560   
2017-06-08  0.01120724 -0.00244577  0.00014545  0.00343643  0.00534642   
2017-06-09 -0.02255493 -0.03877670  0.01323444  0.01220034  0.01285907   
2017-06-12 -0.01571514 -0.02389582 -0.00200947 -0.00253753 -0.00684283   

ticker             AIG        AMAT   

[250 rows x 99 columns]
[[ 4.14364765e-04  4.49317865e-05  3.66980660e-05 ...  9.92392079e-06
   7.59556775e-05  2.40772226e-05]
 [ 4.49317865e-05  1.29559350e-04  1.23195126e-05 ...  1.34110252e-05
   3.92369344e-05  2.42887982e-06]
 [ 3.66980660e-05  1.23195126e-05  1.41717425e-04 ...  1.73657302e-05
   2.73418602e-05  2.46556033e-05]
 ...
 [ 9.92392079e-06  1.34110252e-05  1.73657302e-05 ...  8.93382754e-05
  -6.33642153e-07  3.60171054e-06]
 [ 7.59556775e-05  3.92369344e-05  2.73418602e-05 ... -6.33642153e-07
   4.91677443e-04  2.20560882e-05]
 [ 2.40772226e-05  2.42887982e-06  2.46556033e-05 ...  3.60171054e-06
   2.20560882e-05  8.82645887e-05]]
var60255
[Inequality(Constant(CONSTANT, ZERO, ())), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]
QuadForm(var60255, [[ 4.14364765e-04  4.49317865e-05  3.66980660e-05 ...  9.92392079e-06
   7.59556775e-05  2.40772226e-05]
 [ 4.49317865e-05  1.29559350e-04  1.23195126e-05 ...  1.34110252e-05
   3.92369344e

## Portfolio Turnover
With the portfolio rebalanced, we need to use a metric to measure the cost of rebalancing the portfolio. Implement `get_portfolio_turnover` to calculate the annual portfolio turnover. We'll be using the formulas used in the classroom:

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} * NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ Where $ x_{t,n} $ are the weights at time $ t $ for equity $ n $.

$ SumTotalTurnover $ is just a different way of writing $ \sum \left | x_{t_1,n} - x_{t_2,n} \right | $

In [68]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    #TODO: Implement function
    weight_flip = np.diff(np.flip(all_rebalance_weights, axis=0), axis=0)
    total_turnover = np.abs(weight_flip).sum()
    num_rebalance_events = n_trading_days_in_year // shift_size
    turnover = (total_turnover / rebalance_count) * num_rebalance_events

    return turnover

project_tests.test_get_portfolio_turnover(get_portfolio_turnover)

Tests Passed


Run the following cell to get the portfolio turnover from  `get_portfolio turnover`.

In [69]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

16.594080015192297


That's it! You've built a smart beta portfolio in part 1 and did portfolio optimization in part 2. You can now submit your project.

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.